In [ ]:
from datasets import load_dataset

# Load IMDB (binary sentiment: pos/neg)
dataset = load_dataset("imdb")

# Check structure
print(dataset['train'][0])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [ ]:
!pip install -U datasets huggingface_hub fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.7.0
    Uninstalling fsspec-2025.7.0:
      Successfully uninstalled fsspec-2025.7.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.7.0 requires fsspec==2025.7.0, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [ ]:
from transformers import BertTokenizer

#import BERT's language and tokenizing the rules
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

#ensures all sentences are padded to same length for batching and truncates any reviews which exceed the length of 256 tokens
def tokenizer_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=256)

#applies the tokenizer to all samples in the dataset
tokenized_dataset = dataset.map(tokenizer_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
#Hugging Face Datasets default to dictionary format. We convert them to PyTorch tensors for training.
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

#input_id = encoded ID's, attention_mask = BERT classifies tokens into actual input vs padding, label = actual output 0/1(positive/negative)
train_dataset = tokenized_dataset["train"]
test_dataset = tokenized_dataset["test"]


In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer

# Load model and tokenizer
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

print("Model and tokenizer loaded successfully!")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model and tokenizer loaded successfully!


In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to='none'  # Optional: avoids logging errors if no tracking system is configured
)

#TrainingArguments: Hyperparameters and configs for training.

#output_dir: Where to save model checkpoints.

#evaluation_strategy="epoch": Evaluate on test set after each epoch.

#learning_rate: Smaller is better for BERT (too high = catastrophic forgetting).

#batch_size: Number of samples per forward/backward pass.

#epochs=3: Typical fine-tuning takes 2–4 epochs.

#weight_decay: Regularization to avoid overfitting.

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

Step,Training Loss
500,0.400500
1000,0.318900
1500,0.314900
2000,0.291900
2500,0.292000
3000,0.295100
3500,0.224600
4000,0.193000
4500,0.188600
5000,0.185800


TrainOutput(global_step=9375, training_loss=0.2013523583984375, metrics={'train_runtime': 3812.4735, 'train_samples_per_second': 19.672, 'train_steps_per_second': 2.459, 'total_flos': 9866664576000000.0, 'train_loss': 0.2013523583984375, 'epoch': 3.0})

In [ ]:
metrics = trainer.evaluate()
print(metrics)


{'eval_loss': 0.39537355303764343, 'eval_runtime': 379.1939, 'eval_samples_per_second': 65.929, 'eval_steps_per_second': 8.241, 'epoch': 3.0}


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.argmax(torch.tensor(logits), dim=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

trainer.compute_metrics = compute_metrics
metrics = trainer.evaluate()
print(metrics)


{'eval_loss': 0.39537355303764343, 'eval_accuracy': 0.92296, 'eval_f1': 0.9236199238578681, 'eval_precision': 0.9157754010695187, 'eval_recall': 0.9316, 'eval_runtime': 379.3145, 'eval_samples_per_second': 65.908, 'eval_steps_per_second': 8.239, 'epoch': 3.0}


In [ ]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=256)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=1).item()
    return "positive" if prediction == 1 else "negative"

# Try it
review = """i love the jurassic park films, and most of the Jurassic world, but I HATE this new one.
1- absolutely no connection to the world series
I see it as two separate sub series, park and world. there was no noticeable connection to me. No talk about the park, its just the same company making more dinosaurs for no reasons. It shouldn’t be called “jurassic world”. Just call it jurassic rebirth or something. It has a whole new cast, whole new plot, just like the change from park to world.
2-the plot is booty cheeks
what do you mean a million dollar company has EVERYTHING turned off because a snickers bar??? Thats so unrealistic and stupid. A company that big A. Should have better ventilation B. The whole system shouldn’t just turn off and let the animals out due to a reboot.
And dont even get me started on the characters. They are so useless I forgot all of their names. Scarlets acting is so emotionless, and that boat scene talking about her mom and that guys kid added nothing but an implication at the end of the movie to character development and plot. And I absolutely HATE that family, its two movies in one. They were able to go through the forest with only ONE encounter and nearly dieing. The t-rex got a lobotomy, all the dinos got lobotomys. One of them should have died. The plot would have at least been a tad bit better if someone from the family died, preferably the boyfriend. The end also annoyed me. What do you mean you just SAY what your doing with the DNA but you don’t show us what happened?? I don’t want some “its up to your interpretation” ending, I wanna know if they got arrested, if everyone actually got it, I wanna know!! Also the D-rex was the most uneventful “main” dinosaur. It just slowly walked around and ate like one person. It didn’t even fight any dinosaurs. Also, how did it survive? It seems like there were absolutely no dinosaurs near that location and I feel like it should have starved. The last thing is that scene when the paleontologist guy is hanging off the ledge and the bottle shoots away from him. They could have made a plot point of trying to find it and someone gets eaten, but nooo it just has to land right next to him.
3- Isnt this a dinosaur movie?? Where are the dinosaurs??
I just watched Jurassic park three last night, and that had more action and dinosaurs in it then rebirth. That movie was rushed and the script was unfinished (from what I heard). It had a budget half of rebirth. The dilophosaurus had at most 3 seconds on screen, spino had maybe 3 minutes, ankylosaurus had literally a millisecond, velociraptors where blurry in the background for 6 seconds, a sauropod was in the beginning for 6 seconds, mosasaurus had maybe 7 minutes, titanosaurus had 3 minutes, T-rex had maybe 5 minutes, and I’m not counting D-rex and mutadon, they aren’t really even dinosaurs. Heres the thing, the dinosaurs actually did something and gave substance to the plot in other movies. In the first one, the dilophosaurus has 40 seconds of screen time yet kills someone, and adds plot. The dilophosaurus just yelled for a second and fly away after doing so. All the dinosaurs don’t do anything! Thats what really gets me. They don’t eat anyone, they do nothing but look pretty!!

The only good thing I can say about this film is it had good visuals, thats it."""

print(predict_sentiment(review))


negative
